# 실습1_뉴스데이터 수집하기
- RSS 피드: 웹사이트에 새로 올라오는 글이나 콘텐츠의 요약 정보를 자동으로 제공하는 데이터 포멧
- 직접 사이트에 방문하지 않아도 새 소식을 한눈에 받아볼 수 있음
- 뉴스 -> XML 문서 형식 -> xml 파서(lxml) 필요

## SBS 스포츠 뉴스 데이터 수집하기

In [5]:
# 필요 라이브러리 설치
!pip install lxml
!pip install pandas

프로그램 소스 정리
- <item>
    - <title> : 기사 제목 
    - <link> : 해당 기사의 URL

-> 이제 링크에 연결된 페이지에서 가져올 내용 
본문: #container > div.w_inner > div.w_article > div.w_article_cont > div.w_article_left > div.article_cont_area > div.main_text > div

스크랩핑 시작

In [6]:
import requests
from bs4 import BeautifulSoup
news_rss = requests.get('https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=09&plink=RSSREADER')
print(news_rss) # status check
news_rss_soup = BeautifulSoup(news_rss.content, 'xml')
# 링크 개수 확인 
link_list = news_rss_soup.select('item > link')
print("기사 개수: ", len(link_list))
print(link_list)
#제목 개수 확인, 제목 리스트 만들기 
title_list = news_rss_soup.select('item > title')
print("title 개수: ", len(title_list))
title_list = [title.text for title in title_list] # save title on list 
print(title_list)

<Response [200]>
기사 개수:  29
[<link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196431&amp;plink=RSSLINK&amp;cooper=RSSREADER</link>, <link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196388&amp;plink=RSSLINK&amp;cooper=RSSREADER</link>, <link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196381&amp;plink=RSSLINK&amp;cooper=RSSREADER</link>, <link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196382&amp;plink=RSSLINK&amp;cooper=RSSREADER</link>, <link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196136&amp;plink=RSSLINK&amp;cooper=RSSREADER</link>, <link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196115&amp;plink=RSSLINK&amp;cooper=RSSREADER</link>, <link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196046&amp;plink=RSSLINK&amp;cooper=RSSREADER</link>, <link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196045&amp;plink=RSSLINK&amp;cooper=RSSREADER</link>, <link>https://news.sbs.co.kr/news/endPage.do?news_id=N1008196044&amp;plink=

## 뉴스 본문 스크랩핑 

#container > div.w_inner > div.w_article > div.w_article_cont > div.w_article_left > div.article_cont_area > div.main_text > div

In [7]:
news_data = []
for link in link_list:
    news_response = requests.get(link.text)
    news_content_soup = BeautifulSoup(news_response.content, 'html.parser')
    news_content = news_content_soup.select_one('div.main_text > div')
   
    # 텍스트만 추출해서 저장 (None 처리 포함)
    if news_content:
        cleaned_text = news_content.get_text(separator=' ', strip=True)
        cleaned_text = cleaned_text.replace('▲', '').strip()
    else:
        cleaned_text = ""
        
    news_data.append(cleaned_text)

import pandas as pd
news_df = pd.DataFrame(data={'title': title_list, 'content': news_data})
news_df.to_csv("news.csv", encoding="utf-8-sig", index=False)
print("Save complete")

Save complete
